In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import roc_curve, auc, confusion_matrix
import seaborn as sns
from sklearn.preprocessing import label_binarize
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

IMG_HEIGHT = 256  # Increased to avoid shrinking too much
IMG_WIDTH = 256
BATCH_SIZE = 32

train_dir = '/kaggle/input/brain-tumor-mri-dataset/Training'
test_dir = '/kaggle/input/brain-tumor-mri-dataset/Testing'

train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

base_model_densenet = DenseNet121(weights=None, include_top=False, input_shape=(IMG_HEIGHT, IMG_WIDTH, 3))
base_model_densenet.load_weights('/kaggle/input/densenet/densenet121_weights_tf_dim_ordering_tf_kernels_notop.h5')  # Manually downloaded weights

cnn_model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)),
    layers.MaxPooling2D(2, 2),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(4, activation='softmax')
])

combined_input = layers.Input(shape=(IMG_HEIGHT, IMG_WIDTH, 3))

densenet_output = base_model_densenet(combined_input)
densenet_output = layers.GlobalAveragePooling2D()(densenet_output)
densenet_output = layers.Dense(512, activation='relu')(densenet_output)

cnn_output = cnn_model(combined_input)

merged_output = layers.concatenate([densenet_output, cnn_output])

final_output = layers.Dense(4, activation='softmax')(merged_output)

model = models.Model(inputs=combined_input, outputs=final_output)

model.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

callbacks = [
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
    ModelCheckpoint('brain_tumor_cnn_best_model.keras', save_best_only=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)
]

steps_per_epoch = 60
validation_steps = 60

history = model.fit(
    train_generator,
    epochs=25,  # Set the desired number of epochs
    steps_per_epoch=steps_per_epoch,  # Set steps per epoch to 80
    validation_data=test_generator,
    validation_steps=validation_steps,  # Set validation steps to 80
    verbose=1,
    callbacks=callbacks
)

model.save('/kaggle/working/brain_tumor_cnn_model_enhanced.h5')

model.save_weights('/kaggle/working/brain_tumor_cnn_weights_enhanced.weights.h5')

test_loss, test_acc = model.evaluate(test_generator)
print(f"Test loss: {test_loss}, Test accuracy: {test_acc}")

model.save('brain_tumor_cnn_saved_model_enhanced')

true_labels = test_generator.classes
predictions = model.predict(test_generator, verbose=1)

true_labels_one_hot = label_binarize(true_labels, classes=[0, 1, 2, 3])

fpr, tpr, _ = roc_curve(true_labels_one_hot.ravel(), predictions.ravel())
roc_auc = auc(fpr, tpr)

plt.figure(figsize=(8, 8))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc='lower right')
plt.savefig('roc_curve.png')
plt.show()

cm = confusion_matrix(true_labels, np.argmax(predictions, axis=1))

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=test_generator.class_indices.keys(), yticklabels=test_generator.class_indices.keys())
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.savefig('confusion_matrix.png')
plt.show()

plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

# Save the plot as a file
plt.savefig('accuracy_loss_plot_enhanced.png')
plt.show()

Found 5712 images belonging to 4 classes.
Found 1311 images belonging to 4 classes.


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2             │ (None, 256, 256, 3)    │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ densenet121 (Functional)  │ (None, 8, 8, 1024)     │      7,037,504 │ input_layer_2[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ global_average_pooling2d  │ (None, 1024)           │              0 │ densenet121[0][0]      │
│ (GlobalAveragePooling2D)  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_2 (Dense)           │ (None, 512)            │        524,800 │ global_average_poolin… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ sequential (Sequential)   │ (None, 4)              │     31,510,084 │ input_layer_2[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate (Concatenate) │ (None, 516)            │              0 │ dense_2[0][0],         │
│                           │                        │                │ sequential[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_3 (Dense)           │ (None, 4)              │          2,068 │ concatenate[0][0]      │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 39,074,456 (149.06 MB)

...

/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 19s/step - accuracy: 0.7184 - loss: 0.7109 

/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


60/60 ━━━━━━━━━━━━━━━━━━━━ 1356s 21s/step - accuracy: 0.7203 - loss: 0.7066 - val_accuracy: 0.5545 - val_loss: 1.1898 - learning_rate: 1.0000e-04
Epoch 2/25
60/60 ━━━━━━━━━━━━━━━━━━━━ 1373s 23s/step - accuracy: 0.8944 - loss: 0.2778 - val_accuracy: 0.6674 - val_loss: 1.1416 - learning_rate: 1.0000e-04
Epoch 3/25
60/60 ━━━━━━━━━━━━━━━━━━━━ 1314s 22s/step - accuracy: 0.9629 - loss: 0.1186 - val_accuracy: 0.8002 - val_loss: 0.6023 - learning_rate: 1.0000e-04
Epoch 4/25
60/60 ━━━━━━━━━━━━━━━━━━━━ 1329s 22s/step - accuracy: 0.9519 - loss: 0.1326 - val_accuracy: 0.9016 - val_loss: 0.2589 - learning_rate: 1.0000e-04
Epoch 5/25
60/60 ━━━━━━━━━━━━━━━━━━━━ 1332s 22s/step - accuracy: 0.9621 - loss: 0.1139 - val_accuracy: 0.7605 - val_loss: 0.7207 - learning_rate: 1.0000e-04
Epoch 6/25
60/60 ━━━━━━━━━━━━━━━━━━━━ 1306s 22s/step - accuracy: 0.9695 - loss: 0.0985 - val_accuracy: 0.9451 - val_loss: 0.1654 - learning_rate: 1.0000e-04
Epoch 7/25
60/60 ━━━━━━━━━━━━━━━━━━━━ 1342s 22s/step - accuracy: 0.97

ValueError: Invalid filepath extension for saving. Please add either a `.keras` extension for the native Keras format (recommended) or a `.h5` extension. Use `model.export(filepath)` if you want to export a SavedModel for use with TFLite/TFServing/etc. Received: filepath=brain_tumor_cnn_saved_model_enhanced.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
from sklearn.metrics import f1_score, accuracy_score, classification_report, confusion_matrix, roc_curve, auc, precision_recall_curve
import seaborn as sns
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Paths to your dataset directories
train_dir = '/kaggle/input/brain-tumor-mri-dataset/Training'
test_dir = '/kaggle/input/brain-tumor-mri-dataset/Testing'

# Parameters
img_height, img_width = 256, 256
batch_size = 32

# Data Preprocessing
train_datagen = ImageDataGenerator(rescale=1.0 / 255.0)
test_datagen = ImageDataGenerator(rescale=1.0 / 255.0)

# Load test dataset
test_data = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

# Extract test images and labels
X_test = np.concatenate([test_data[i][0] for i in range(len(test_data))], axis=0)
y_test = np.concatenate([test_data[i][1] for i in range(len(test_data))], axis=0)

# Load the trained model
model = load_model('/kaggle/working/brain_tumor_cnn_model_enhanced.h5')

# Make predictions
y_pred_probs = model.predict(X_test)

# For multi-class classification
if y_test.shape[1] > 1:
    y_pred = np.argmax(y_pred_probs, axis=1)
    y_true = np.argmax(y_test, axis=1)
else:  # For binary classification
    y_pred = (y_pred_probs > 0.5).astype(int).flatten()
    y_true = y_test.flatten()

# Calculate Accuracy
accuracy = accuracy_score(y_true, y_pred)

# Calculate F1 Score
f1 = f1_score(y_true, y_pred, average='binary' if y_test.shape[1] == 1 else 'macro')

# Classification Report
target_names = list(test_data.class_indices.keys())
report = classification_report(y_true, y_pred, target_names=target_names)

# Confusion Matrix
conf_matrix = confusion_matrix(y_true, y_pred)

# Per-class Accuracy
classified_accuracy = conf_matrix.diagonal() / conf_matrix.sum(axis=1)

# Display metrics
print(f"Accuracy: {accuracy:.4f}")
print(f"F1 Score: {f1:.4f}")
print("\nClassification Report:\n", report)
print("\nClassified Accuracy (Per-class):", classified_accuracy)

# Create a single figure with subplots for multiple graphs
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
axes = axes.flatten()

# 1. Confusion Matrix
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=target_names, yticklabels=target_names, cbar=False, ax=axes[0])
axes[0].set_xlabel('Predicted Labels', fontsize=14)
axes[0].set_ylabel('True Labels', fontsize=14)
axes[0].set_title('Confusion Matrix', fontsize=16)
axes[0].tick_params(axis='both', labelsize=12)
axes[0].grid(True)

# 2. ROC Curve (only for binary classification)
if y_test.shape[1] == 1:
    # For binary classification, extract probabilities for the positive class (class 1)
    y_pred_probs_binary = y_pred_probs[:, 0] if y_pred_probs.ndim > 1 else y_pred_probs

    fpr, tpr, thresholds = roc_curve(y_true, y_pred_probs_binary)
    roc_auc = auc(fpr, tpr)

    axes[1].plot(fpr, tpr, color='darkorange', lw=3, label=f'ROC curve (AUC = {roc_auc:.2f})')
    axes[1].plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    
    axes[1].set_xlim([0.0, 1.0])
    axes[1].set_ylim([0.0, 1.05])
    axes[1].set_xticks(np.arange(0, 1.1, step=0.05))
    axes[1].set_yticks(np.arange(0, 1.1, step=0.05))
    axes[1].set_xlabel('False Positive Rate', fontsize=14)
    axes[1].set_ylabel('True Positive Rate', fontsize=14)
    axes[1].set_title('Receiver Operating Characteristic (ROC)', fontsize=16)
    axes[1].legend(loc='lower right', fontsize=12)
    axes[1].grid(True)
else:
    # For multi-class classification (One-vs-Rest ROC curve)
    n_classes = y_test.shape[1]
    
    fpr = {}
    tpr = {}
    roc_auc = {}
    
    for i in range(n_classes):
        fpr[i], tpr[i], _ = roc_curve(y_test[:, i], y_pred_probs[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])
        
    for i in range(n_classes):
        axes[1].plot(fpr[i], tpr[i], lw=3, label=f'Class {target_names[i]} (AUC = {roc_auc[i]:.2f})')

    axes[1].plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    
    axes[1].set_xlim([0.0, 1.0])
    axes[1].set_ylim([0.0, 1.05])
    axes[1].set_xticks(np.arange(0, 1.1, step=0.05))
    axes[1].set_yticks(np.arange(0, 1.1, step=0.05))
    axes[1].set_xlabel('False Positive Rate', fontsize=14)
    axes[1].set_ylabel('True Positive Rate', fontsize=14)
    axes[1].set_title('Receiver Operating Characteristic (ROC) - Multi-Class', fontsize=16)
    axes[1].legend(loc='lower right', fontsize=12)
    axes[1].grid(True)

# 3. Precision-Recall Curve
if y_test.shape[1] > 1:  # Multi-class classification
    for i, class_name in enumerate(target_names):
        precision, recall, _ = precision_recall_curve(y_test[:, i], y_pred_probs[:, i])
        axes[2].plot(recall, precision, lw=3, label=f'Class {class_name}')
    axes[2].set_xlabel('Recall', fontsize=14)
    axes[2].set_ylabel('Precision', fontsize=14)
    axes[2].set_title('Precision-Recall Curve (Multi-Class)', fontsize=16)
else:  # Binary classification
    precision, recall, _ = precision_recall_curve(y_true, y_pred_probs)
    axes[2].plot(recall, precision, color='blue', lw=3)
    axes[2].set_xlabel('Recall', fontsize=14)
    axes[2].set_ylabel('Precision', fontsize=14)
    axes[2].set_title('Precision-Recall Curve', fontsize=16)

axes[2].set_xlim([0.0, 1.0])
axes[2].set_ylim([0.0, 1.0])
axes[2].set_xticks(np.arange(0, 1.1, step=0.05))
axes[2].set_yticks(np.arange(0, 1.1, step=0.05))
axes[2].grid(True)
axes[2].legend(loc='lower left', fontsize=12)

# Adjust layout
fig.tight_layout()
plt.show()

Found 1311 images belonging to 4 classes.
41/41 ━━━━━━━━━━━━━━━━━━━━ 165s 4s/step
Accuracy: 0.9969
F1 Score: 0.9967

Classification Report:
               precision    recall  f1-score   support

      glioma       1.00      0.99      0.99       300
  meningioma       0.99      1.00      0.99       306
     notumor       1.00      1.00      1.00       405
   pituitary       1.00      1.00      1.00       300

    accuracy                           1.00      1311
   macro avg       1.00      1.00      1.00      1311
weighted avg       1.00      1.00      1.00      1311


Classified Accuracy (Per-class): [0.99333333 0.99673203 1.         0.99666667]
